In [1]:
import glob, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

legislatura = 'Legislatura XLVIII'


## Dimensiones

### Asistencia
Presiencia en el plenario y comisiones<br>
Promedio de la presencia en el plenario y las comisiones (asistencias / citaciones)

In [2]:
asistencia_plenario_df = pd.read_csv('data/%s/asistencia_plenario.csv' % legislatura)
asistencia_comision_df = pd.read_csv('data/%s/asistencia_comisiones.csv' % legislatura)
asistencia_df = pd.concat([asistencia_plenario_df[['id_senador', 'asistencias', 'citaciones']], asistencia_comision_df[['id_senador', 'asistencias', 'citaciones']]])
asistencia_df = asistencia_df.groupby('id_senador').sum()

# Cálculo de z-score de asistencias
asistencia_df.loc[:,'asistencia_med'] = asistencia_df.apply(lambda x : x['asistencias'] * 1.0 / x['citaciones'], axis=1)
asistencia_media = asistencia_df['asistencia_med'].mean()
asistencia_std = asistencia_df['asistencia_med'].std()
asistencia_df.loc[:,'asistencia_zidx'] = asistencia_df.apply(lambda x : (x['asistencia_med'] * 1.0 - asistencia_media) / asistencia_std, axis=1)
asistencia_df.reset_index(inplace=True)
asistencia_df.head()

,id_senador,asistencias,citaciones,asistencia_med,asistencia_zidx
0,196,176,225,0.782222,-0.521744
1,197,502,597,0.840871,-0.131068
2,260,171,190,0.900000,0.262807
3,277,359,439,0.817768,-0.284966
4,290,27,36,0.750000,-0.736386


## Proyectos
Incluye las siguientes actividades:
* Presentación o firma de proyectos de ley
* Proyectos de resolución
* Proyectos de minutas
* Proyectos de declaración de cuerpo
<br><br>El índice se basa en la suma de todos los proyectos presentados

In [3]:
proyectos_presentados_df = pd.read_csv('data/%s/proyectos_presentados.csv' % legislatura)
proyectos_df = pd.read_csv('data/%s/proyectos.csv' % legislatura)
proyectos_presentados_df = proyectos_presentados_df.merge(proyectos_df, on='id_proyecto', how='left')

proyectos_presentados_df = proyectos_presentados_df[proyectos_presentados_df['tipo'].isin(['PROYECTO DE DECLARACION', 'PROYECTO DE LEY', 'PROYECTO DE MINUTA DE COMUNICACION','PROYECTO DE RESOLUCION'])]
proyectos_presentados_df = proyectos_presentados_df[['id_senador', 'id_proyecto']].groupby('id_senador').size().reset_index(name='proyectos_total')

proyectos_presentados_media = proyectos_presentados_df['proyectos_total'].mean()
proyectos_presentados_std = proyectos_presentados_df['proyectos_total'].std()
proyectos_presentados_df.loc[:,'proyectos_presentados_zidx'] = proyectos_presentados_df.apply(lambda x : (x['proyectos_total'] * 1.0 - proyectos_presentados_media) / proyectos_presentados_std, axis=1)
proyectos_presentados_df.head()

,id_senador,proyectos_total,proyectos_presentados_zidx
0,196,19,2.112014
1,197,9,0.419799
2,260,8,0.250578
3,277,2,-0.764751
4,290,1,-0.933972


## Informante en comisión
Informantes de proyectos en las comisiones en las que participa<br>
El índice se basa en la suma de número de acciones

In [4]:
actuacion_parlamentaria_df = pd.read_csv('data/%s/actuacion_parlamentaria.csv' % legislatura, index_col=0)
informante_comision_df = actuacion_parlamentaria_df[actuacion_parlamentaria_df['tipo'] == 'INFORMA']
informante_comision_df = informante_comision_df.groupby('id_senador').size().reset_index(name='informes_total')
informante_comision_media = informante_comision_df['informes_total'].mean()
informante_comision_std = informante_comision_df['informes_total'].std()
informante_comision_df.loc[:,'informe_comision_zidx'] = informante_comision_df.apply(lambda x : (x['informes_total'] * 1.0 - informante_comision_media) / informante_comision_std, axis=1)
informante_comision_df.head()


,id_senador,informes_total,informe_comision_zidx
0,196,1,-0.660309
1,197,27,0.903053
2,260,9,-0.179274
3,277,13,0.061243
4,327,15,0.181501


## Debate
Interviene en debates sobre proyectos<br>
El índice se basa en la suma de las intervenciones en el plenario

In [5]:
interviene_df = actuacion_parlamentaria_df[actuacion_parlamentaria_df['tipo'] == 'INTERVIENE']
interviene_df = interviene_df.groupby('id_senador').size().reset_index(name='interveciones_total')
interviene_media = interviene_df['interveciones_total'].mean()
interviene_std = interviene_df['interveciones_total'].std()
interviene_df.loc[:,'intervenciones_zidx'] = interviene_df.apply(lambda x : (x['interveciones_total'] * 1.0 - interviene_media) / interviene_std, axis=1)
interviene_df.head()


,id_senador,interveciones_total,intervenciones_zidx
0,196,131,0.586785
1,197,324,2.460087
2,260,230,1.547702
3,277,117,0.450898
4,290,19,-0.500312


## Exposiciones
Escribe notas o realiza exposiciones sobre temas determinados.<br>
El índice se basa en la suma de las exposiciones realizadas

In [6]:
expone_df = actuacion_parlamentaria_df[actuacion_parlamentaria_df['tipo'] == 'EXPONE']
expone_df = expone_df.groupby('id_senador').size().reset_index(name='exposiciones_total')
expone_media = expone_df['exposiciones_total'].mean()
expone_std = expone_df['exposiciones_total'].std()
expone_df.loc[:,'exposiciones_zidx'] = expone_df.apply(lambda x : (x['exposiciones_total'] * 1.0 - expone_media) / expone_std, axis=1)
expone_df.head()


,id_senador,exposiciones_total,exposiciones_zidx
0,196,22,0.764654
1,197,23,0.837327
2,260,5,-0.470799
3,277,19,0.546632
4,327,28,1.200696


## Pedidos de informe
Solicita un pedido de informes al gobierno.<br>
El índice se basa en la suma de pedidos de informe

In [7]:
pedidos_de_informe_df = pd.read_csv('data/%s/pedidos_informe.csv' % legislatura, index_col=0)
pedidos_de_informe_df = pedidos_de_informe_df[['id_senador', 'id_proyecto']].groupby('id_senador').size().reset_index(name='pedidos_informe_total')
pedidos_de_informe_media = pedidos_de_informe_df['pedidos_informe_total'].mean()
pedidos_de_informe_std = pedidos_de_informe_df['pedidos_informe_total'].std()
pedidos_de_informe_df.loc[:,'pedidos_informe_zidx'] = pedidos_de_informe_df.apply(lambda x : (x['pedidos_informe_total'] * 1.0 - pedidos_de_informe_media) / pedidos_de_informe_std, axis=1)
pedidos_de_informe_df.head()


,id_senador,pedidos_informe_total,pedidos_informe_zidx
0,196,12,-0.183980
1,197,7,-0.533693
2,260,14,-0.044094
3,327,40,1.774417
4,408,2,-0.883407


In [8]:
indice_senadores_df = asistencia_df[['id_senador', 'asistencia_zidx']].merge(proyectos_presentados_df[['id_senador', 'proyectos_presentados_zidx']], on='id_senador')
indice_senadores_df = indice_senadores_df.merge(informante_comision_df[['id_senador', 'informe_comision_zidx']], on='id_senador')
indice_senadores_df = indice_senadores_df.merge(interviene_df[['id_senador', 'intervenciones_zidx']], on='id_senador')
indice_senadores_df = indice_senadores_df.merge(expone_df[['id_senador', 'exposiciones_zidx']], on='id_senador')
indice_senadores_df = indice_senadores_df.merge(pedidos_de_informe_df[['id_senador', 'pedidos_informe_zidx']], on='id_senador')
indice_senadores_df.loc[:, 'zidx'] = indice_senadores_df['asistencia_zidx'] +indice_senadores_df['asistencia_zidx'] + indice_senadores_df['proyectos_presentados_zidx'] +indice_senadores_df['informe_comision_zidx'] + indice_senadores_df['intervenciones_zidx'] + indice_senadores_df['exposiciones_zidx'] + indice_senadores_df['pedidos_informe_zidx']
indice_senadores_df.sort_values('zidx', ascending=False).head()



,id_senador,asistencia_zidx,proyectos_presentados_zidx,informe_comision_zidx,intervenciones_zidx,exposiciones_zidx,pedidos_informe_zidx,zidx
6,922,0.270127,1.435128,1.203699,5.274892,2.726843,1.774417,12.955233
3,327,0.094903,1.942793,0.181501,3.090991,1.200696,1.774417,8.380204
20,7740,0.290182,0.419799,-0.600179,2.285375,3.090211,1.354760,7.130330
4,408,0.637333,-0.933972,4.871586,1.635058,0.691980,-0.883407,6.655910
18,6900,-0.092796,0.419799,-0.299533,0.557667,4.979727,0.935104,6.407173
